# Applying ML methods to Records objects

In [25]:
# conda install folium

In [3]:
import records
import toyplot
import folium
import pandas as pd
import numpy as np


### Load data and filter it
The Epochs call is commented out below to instead use the `load_epochs_from_csv()` function, which is faster when the data has already been downloaded. This code raises a warning (which is not an error) and recommends a change to our code in Records that would avoid the warning. When reading a dataframe this big it wants you do tell it what the datatype will be so it can load faster. We'll skip that for now. 

In [6]:
# get Bombus records
#ep = records.Epochs("Bombus", 1900, 2010, 10)
ep = records.load_epochs_from_csv(
    "/home/deren/PDSB/records/data/Bombus-1900-2015-df.csv",
    )

/home/deren/miniconda2/envs/Py36/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (1,2,3,4,6,7,13,20,29,30,35,38,39,44,45,52,54,55,56,57,60,61,63,64,65,70,71,85,86,87,91,92,93,96,99,100,101,102,106,110,114,115,116,117,120,122,132,134,135,136,138,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### The data set 
Select just the columns we plan to analyze. 

In [7]:
# select columns
cols = [
    "specificEpithet", 
    "decimalLatitude", 
    "decimalLongitude", 
    "stateProvince",
    "year", 
    "elevation",
]

# rename cols
data = ep.df[cols]
data.columns = ["species", "lat", "long", "state", "year", "elevation"]
data.head(10)

,species,lat,long,state,year,elevation
0,fervidus,38.77750,-95.18750,Kansas,1900,NaN
1,fervidus,38.77750,-95.18750,Kansas,1900,NaN
2,vosnesenskii,37.42740,-122.16980,California,1900,NaN
3,affinis,43.39812,-76.47744,New York,1900,NaN
4,fervidus,38.77750,-95.18750,Kansas,1900,NaN
5,pensylvanicus,41.04650,-100.74700,Nebraska,1900,NaN
6,fervidus,38.77750,-95.18750,Kansas,1900,NaN
7,variabilis,39.27972,-89.88167,Illinois,1900,NaN
8,fervidus,38.77750,-95.18750,Kansas,1900,NaN
9,sylvicola,64.50111,-165.40639,Alaska,1900,NaN


### Filter for NaN and remove taxa that occurred rarely

In [8]:
# filter out any rows that contain NaN
df = data[np.invert(np.any(data.isna(), axis=1))]

# filter out records for species that occur < 500 times
spc = df.species.value_counts()
df = df[[spc[spp] > 500 for spp in df.species]]

# shuffle row order
df = df.sample(frac=1).reset_index(drop=True)

In [9]:
# the data set now includes 20 species
df.species.unique().shape

(20,)

In [10]:
# show data
df.head()

,species,lat,long,state,year,elevation
0,bifarius,64.858500,-147.817700,Alaska,2010,153.0
1,huntii,37.591300,-112.257500,Utah,2008,2393.0
2,flavifrons,37.642300,-105.222000,Colorado,1976,2743.0
3,bifarius,46.209030,-117.773520,Washington,1955,1524.0
4,balteatus,39.010269,-107.042545,Colorado,1970,3200.0


### Split data into features and labels

In [11]:
X = df[["lat", "long", "elevation"]]
y = df["species"]

X.shape

(41459, 3)

### Fit a GaussianNB model

In [12]:
# next we measure the fit?
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1)

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(Xtrain, ytrain)
y_model = model.predict(Xtest)

In [14]:
# measure accuracy of prediction for real data
from sklearn.metrics import accuracy_score
accuracy_score(ytest, y_model)

0.22643511818620357

In [15]:
dat = {
    "inferred": y_model,
    "actual": ytest,
}
dat.update(Xtest)
pdf = pd.DataFrame(dat)

In [28]:
pdf.head(10)

,actual,elevation,inferred,lat,long
36397,jonellus,701.04,jonellus,64.883333,-148.05000
24449,vosnesenskii,991.00,vosnesenskii,40.877660,-121.56048
4528,bifarius,393.00,jonellus,63.933100,-145.35750
40069,centralis,2822.00,bifarius,35.331160,-111.71185
11725,bifarius,1377.00,vosnesenskii,45.325140,-121.63603
8924,frigidus,136.00,jonellus,64.849030,-147.82957
40892,sylvicola,193.00,jonellus,65.346300,-148.28740
37675,flavifrons,153.00,jonellus,64.858500,-147.81770
33824,occidentalis,1768.00,huntii,40.094000,-105.34700
10008,rufocinctus,2079.00,bifarius,45.061800,-111.25550


### Examine confusion matrix
This tells us which taxa were most often mixed up in predictions. 

In [18]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(
    ytest,
    y_model,
    labels=ytest.unique(),
).astype(float)

In [19]:
# get matrix as a proportion of correct predictions for each taxon
props = (mat) / (mat.sum(axis=1))[:, None]

In [30]:
# add labels to left side
llocator = toyplot.locator.Explicit(
    range(props.shape[0]),
    ytest.unique(),
)
toyplot.matrix(props, llocator=llocator, tlabel="Confusion matrix");

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 jonellus 0.96178343949 0.0 0.0 0.0 0.031847133758 0.0 0.00636942675159 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 vosnesenskii 0.0 0.917045454545 0.0772727272727 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.00113636363636 0.0 0.0 0.0 0.00454545454545 0.0 0.0 0.0 0.0 0.0 bifarius 0.0196516301921 0.190263510496 0.481911567664 0.0125055828495 0.0 0.0 0.0 0.0 0.00937918713711 0.0 0.00669941938365 0.0 0.0 0.0 0.279589102278 0.0 0.0 0.0 0.0 0.0 centralis 0.0 0.010752688172 0.458064516129 0.0172043010753 0.0 0.0 0.0 0.0 0.00215053763441 0.0 0.0193548387097 0.0 0.0 0.0 0.49247311828 0.0 0.0 0.0 0.0 0.0 frigidus 0.642857142857 0.0 0.0931677018634 0.0217391304348 0.0341614906832 0.0 0.0 0.0124223602484 0.0745341614907 0.0 0.0 0.0 0.0 0.0 0.121118012422 0.0 0.0 0.0 0.0 0.0 sylvicola 0.348336594912 0.00782778864971 0.191780821918 0.111545988258 0.00978473581213 0.0 0.00391389432485 0.0156555772994 0.135029354207 0.0 0.0 0.0 0.0 0.0 0.176125244618 0.0 0.0 0.0 0.0 0.0 flavifrons 0.348056537102 0.183745583039 0.177561837456 0.0441696113074 0.000883392226148 0.0 0.000883392226148 0.0194346289753 0.0132508833922 0.0 0.00265017667845 0.000883392226148 0.0 0.0388692579505 0.16961130742 0.0 0.0 0.0 0.0 0.0 occidentalis 0.479 0.079 0.172 0.019 0.002 0.0 0.002 0.003 0.004 0.0 0.003 0.0 0.0 0.0 0.237 0.0 0.0 0.0 0.0 0.0 rufocinctus 0.0 0.0275 0.295 0.01 0.0 0.0 0.0 0.0 0.01 0.0 0.01 0.0 0.0 0.0 0.6475 0.0 0.0 0.0 0.0 0.0 melanopygus 0.159090909091 0.347402597403 0.175324675325 0.0324675324675 0.00974025974026 0.0 0.012987012987 0.0779220779221 0.107142857143 0.0 0.0 0.012987012987 0.0 0.00649350649351 0.0584415584416 0.0 0.0 0.0 0.0 0.0 balteatus 0.134328358209 0.0 0.044776119403 0.10447761194 0.00995024875622 0.0 0.0 0.00497512437811 0.238805970149 0.0 0.00995024875622 0.0 0.0 0.0 0.452736318408 0.0 0.0 0.0 0.0 0.0 sitkensis 0.0375 0.50625 0.1125 0.0 0.0 0.0 0.03125 0.21875 0.0 0.0 0.0 0.06875 0.0 0.025 0.0 0.0 0.0 0.0 0.0 0.0 insularis 0.129793510324 0.144542772861 0.330383480826 0.00589970501475 0.0 0.0 0.00589970501475 0.0 0.00294985250737 0.0 0.00589970501475 0.0 0.0 0.0 0.374631268437 0.0 0.0 0.0 0.0 0.0 mixtus 0.114485981308 0.435747663551 0.22546728972 0.036214953271 0.0 0.0 0.00233644859813 0.00116822429907 0.035046728972 0.0 0.00116822429907 0.0 0.0 0.0385514018692 0.109813084112 0.0 0.0 0.0 0.0 0.0 huntii 0.0 0.025171624714 0.391304347826 0.00686498855835 0.0 0.0 0.00228832951945 0.0 0.0137299771167 0.0 0.0183066361556 0.0 0.0 0.0 0.54233409611 0.0 0.0 0.0 0.0 0.0 nevadensis 0.0 0.119718309859 0.154929577465 0.0352112676056 0.0 0.0 0.0 0.0 0.0211267605634 0.0 0.0281690140845 0.0 0.0 0.0 0.640845070423 0.0 0.0 0.0 0.0 0.0 californicus 0.0 0.253623188406 0.31884057971 0.0 0.0 0.0 0.0 0.0 0.00724637681159 0.0 0.0 0.0 0.0 0.0 0.420289855072 0.0 0.0 0.0 0.0 0.0 fervidus 0.0 0.0594059405941 0.267326732673 0.00990099009901 0.0 0.0 0.0594059405941 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.60396039604 0.0 0.0 0.0 0.0 0.0 morrisoni 0.0 0.015037593985 0.436090225564 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.00751879699248 0.0 0.0 0.0 0.541353383459 0.0 0.0 0.0 0.0 0.0 appositus 0.0 0.0204081632653 0.361516034985 0.0379008746356 0.0 0.0 0.0 0.0 0.0145772594752 0.0 0.00583090379009 0.0 0.0 0.0 0.559766763848 0.0 0.0 0.0 0.0 0.0 Confusion matrix

### Examine distribution plots of species

In [22]:
def species_map(name, df, maxrecords=100):
    m = folium.Map(
        location=[50,-110],
        zoom_start=2)

    sdata = df[df.inferred==name]
    for i in range(min(sdata.shape[0], maxrecords)):
        folium.Marker(
            popup="inferred to be {}".format(name),
            location=(sdata.iloc[i]['lat'], sdata.iloc[i]['long']),
            ).add_to(m)
        
    sdata = df[df.actual==name]
    for i in range(min(sdata.shape[0], maxrecords)):
        folium.Marker(
            icon=folium.Icon(color='red'),
            popup="true {}".format(name),
            location=(sdata.iloc[i]['lat'], sdata.iloc[i]['long']),
            ).add_to(m)
    return m

In [27]:
species_map("vosnesenskii", pdf, 100)